In [1]:
import gurobipy as gp
from gurobipy import Model, GRB, quicksum
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import numpy as np
import seaborn as sns
from IPython.display import display
from matplotlib.ticker import MaxNLocator
import random



# 参数
m = 15
K = 5
L_levels = [20, 15, 10, 5, 1]  
E_levels = [13, 28, 38, 45, 120] 
E_PU_max = [5500] * m # limit 650
CPU_capacity = [600] * m
f_v_levels = [15, 20, 30, 40, 60]
P_idle = [18] * m
P_max = [36] * m
T_total = 300
total_tasks = 100


chain_tasks = {
    0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    1: [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
    2: [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
    3: [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
    4: [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
    5: [50, 51, 52, 53, 54, 55, 56, 57],
    6: [58, 59, 60],
    7: [61, 62, 63],
    8: [64, 65, 66],
    9: [67, 68, 69],
    10: [70, 71, 72],
    11: [73, 74, 75],
    12: [76, 77, 78],
    13: [79, 80, 81],
    14: [82, 83, 84],
    15: [85, 86, 87],
    16: [88, 89, 90],
    17: [91, 92, 93],
    18: [94, 95, 96],
    19: [97, 98, 99]
}


num_chains = len(chain_tasks)
n = max(max(v) for v in chain_tasks.values()) + 1  # 总任务数
L_max = [len(chain_tasks[k]) * 10 for k in chain_tasks]  # 每条链的最大延迟



alpha_1 = 2500
alpha_2 = 10
alpha_pu = 1


In [2]:
import random

def initialize_population(pop_size, n, m, K):
    population = []
    for i in range(pop_size):
        individual = {
            "task_pu": [random.randint(0, m - 1) for _ in range(n)],
            "task_level": [0 if random.random() < 0.5 else random.randint(0, K - 1) for _ in range(n)]
        }
        population.append(individual)
    return population


def calculate_fitness(individual, params):
    alpha_1, alpha_2, alpha_pu, E_levels, L_levels, f_v_levels, CPU_capacity, \
    P_idle, P_max, T_total, chain_tasks, L_max, E_PU_max, n, m = params

    task_pu = individual["task_pu"]
    task_level = individual["task_level"]

    # 初始化
    fitness = 0
    penalty = 0

    # 判断链激活情况
    z = []
    for chain_id, tasks in chain_tasks.items():
        chain_delay = sum(L_levels[task_level[i]] for i in tasks)
        if chain_delay <= L_max[chain_id]:
            z.append(1)
        else:
            z.append(0)

    total_chain_accept = sum(z)
    fitness += alpha_1 * total_chain_accept

    # 计算任务总能耗（只计算激活链中的任务）
    total_task_energy = sum(
        E_levels[task_level[i]] for chain_id, tasks in chain_tasks.items() if z[chain_id]
        for i in tasks
    )
    fitness -= alpha_2 * total_task_energy

    # 计算PU能耗并进行PU约束检查
    total_pu_energy = 0
    for j in range(m):
        tasks_on_pu = [
            i for i in range(n) 
            if task_pu[i] == j and any(i in tasks and z[chain_id] for chain_id, tasks in chain_tasks.items())
        ]

        execution_time = sum(L_levels[task_level[i]] for i in tasks_on_pu)
        idle_time = T_total - execution_time if T_total >= execution_time else 0

        cpu_usage = sum(f_v_levels[task_level[i]] for i in tasks_on_pu)
        if cpu_usage > CPU_capacity[j]:
            penalty += (cpu_usage - CPU_capacity[j]) * 1000

        pu_e = P_idle[j] * idle_time + (P_max[j] - P_idle[j]) * execution_time
        total_pu_energy += pu_e

        if pu_e > E_PU_max[j]:
            penalty += (pu_e - E_PU_max[j]) * 1000

    fitness -= alpha_pu * total_pu_energy
    fitness -= penalty

    return fitness


# 选择（轮盘赌选择）
def select_parents(population, fitnesses):
    min_fit = min(fitnesses)
    adjusted_fitnesses = [f - min_fit + 1 for f in fitnesses]
    parents = random.choices(population, weights=adjusted_fitnesses, k=2)
    return parents

# 交叉
def crossover(parent1, parent2):
    crossover_point = random.randint(0, len(parent1["task_pu"]) - 1)
    child = {
        "task_pu": parent1["task_pu"][:crossover_point] + parent2["task_pu"][crossover_point:],
        "task_level": parent1["task_level"][:crossover_point] + parent2["task_level"][crossover_point:]
    }
    return child

# 变异
def mutate(individual, m, K, mutation_rate=0.01):
    for i in range(len(individual["task_pu"])):
        if random.random() < mutation_rate:
            individual["task_pu"][i] = random.randint(0, m - 1)
        if random.random() < mutation_rate:
            individual["task_level"][i] = random.randint(0, K - 1)

def genetic_algorithm(n, m, K, generations, pop_size, mutation_rate, params):
    population = initialize_population(pop_size, n, m, K)
    best_fitness = float('-inf')
    best_individual = None

    for gen in range(generations):
        fitnesses = [calculate_fitness(ind, params) for ind in population]
        new_population = []

        max_fit = max(fitnesses)
        if max_fit > best_fitness:
            best_fitness = max_fit
            best_individual = population[fitnesses.index(max_fit)]

        while len(new_population) < pop_size:
            parents = select_parents(population, fitnesses)
            child = crossover(parents[0], parents[1])
            mutate(child, m, K, mutation_rate)
            new_population.append(child)

        population = new_population
        print(f"Generation {gen + 1}: Best Fitness = {best_fitness:.2f}")

    # 生成链激活情况报告
    task_level = best_individual["task_level"]
    z = []
    for chain_id, tasks in chain_tasks.items():
        chain_delay = sum(L_levels[task_level[i]] for i in tasks)
        z.append(1 if chain_delay <= L_max[chain_id] else 0)
    
    print("\n📋 最佳个体链激活状态：")
    for k in range(len(z)):
        status = "✅ 激活" if z[k] else "❌ 未激活"
        print(f"链 {k}: {status}")

    return best_individual, best_fitness, z

def analyze_solution(individual, chain_activation, params):
    alpha_1, alpha_2, alpha_pu, E_levels, L_levels, f_v_levels, CPU_capacity, \
    P_idle, P_max, T_total, chain_tasks, L_max, E_PU_max, n, m = params

    task_pu = individual["task_pu"]
    task_level = individual["task_level"]

    print("\n📊 PU CPU 使用情况：")
    for j in range(m):
        tasks_on_pu = [
            i for i in range(n)
            if task_pu[i] == j and any(i in tasks and chain_activation[chain_id]
                                       for chain_id, tasks in chain_tasks.items())
        ]
        cpu_usage = sum(f_v_levels[task_level[i]] for i in tasks_on_pu)
        print(f"PU-{j}: CPU 使用 {cpu_usage} / 容量 {CPU_capacity[j]}")

    print("\n📊 任务链延迟情况：")
    for chain_id, tasks in chain_tasks.items():
        chain_delay = sum(L_levels[task_level[i]] for i in tasks)
        status = "✅" if chain_activation[chain_id] else "❌（未激活）"
        print(f"链-{chain_id}: 延迟 {chain_delay} / 限制 {L_max[chain_id]} {status}")

    print("\n📊 PU能耗情况：")
    total_pu_energy = 0
    for j in range(m):
        tasks_on_pu = [
            i for i in range(n)
            if task_pu[i] == j and any(i in tasks and chain_activation[chain_id]
                                       for chain_id, tasks in chain_tasks.items())
        ]
        execution_time = sum(L_levels[task_level[i]] for i in tasks_on_pu)
        idle_time = max(0, T_total - execution_time)
        pu_energy = P_idle[j] * idle_time + (P_max[j] - P_idle[j]) * execution_time
        total_pu_energy += pu_energy
        print(f"PU-{j}: 能耗 {pu_energy:.2f} / 上限 {E_PU_max[j]}")

    total_task_energy = sum(
        E_levels[task_level[i]] for chain_id, tasks in chain_tasks.items()
        if chain_activation[chain_id] for i in tasks
    )

    print("\n📊 总体能耗与激活情况：")
    print(f"激活链数: {sum(chain_activation)} / 总链数: {len(chain_activation)}")
    print(f"任务总能耗: {total_task_energy}")
    print(f"PU 总能耗: {total_pu_energy:.2f}")

def analyze_solution(individual, chain_activation, params):
    alpha_1, alpha_2, alpha_pu, E_levels, L_levels, f_v_levels, CPU_capacity, \
    P_idle, P_max, T_total, chain_tasks, L_max, E_PU_max, n, m = params

    task_pu = individual["task_pu"]
    task_level = individual["task_level"]

    print("\n📊 PU CPU 使用情况：")
    for j in range(m):
        tasks_on_pu = [
            i for i in range(n)
            if task_pu[i] == j and any(i in tasks and chain_activation[chain_id]
                                       for chain_id, tasks in chain_tasks.items())
        ]
        cpu_usage = sum(f_v_levels[task_level[i]] for i in tasks_on_pu)
        print(f"PU-{j}: CPU 使用 {cpu_usage} / 容量 {CPU_capacity[j]}")

    print("\n📊 任务链延迟情况：")
    for chain_id, tasks in chain_tasks.items():
        chain_delay = sum(L_levels[task_level[i]] for i in tasks)
        status = "✅" if chain_activation[chain_id] else "❌（未激活）"
        print(f"链-{chain_id}: 延迟 {chain_delay} / 限制 {L_max[chain_id]} {status}")

    print("\n📊 PU能耗情况：")
    total_pu_energy = 0
    for j in range(m):
        tasks_on_pu = [
            i for i in range(n)
            if task_pu[i] == j and any(i in tasks and chain_activation[chain_id]
                                       for chain_id, tasks in chain_tasks.items())
        ]
        execution_time = sum(L_levels[task_level[i]] for i in tasks_on_pu)
        idle_time = max(0, T_total - execution_time)
        pu_energy = P_idle[j] * idle_time + (P_max[j] - P_idle[j]) * execution_time
        total_pu_energy += pu_energy
        print(f"PU-{j}: 能耗 {pu_energy:.2f} / 上限 {E_PU_max[j]}")

    total_task_energy = sum(
        E_levels[task_level[i]] for chain_id, tasks in chain_tasks.items()
        if chain_activation[chain_id] for i in tasks
    )

    print("\n📊 总体能耗与激活情况：")
    print(f"激活链数: {sum(chain_activation)} / 总链数: {len(chain_activation)}")
    print(f"任务总能耗: {total_task_energy}")
    print(f"PU 总能耗: {total_pu_energy:.2f}")





In [3]:
params = [
    14000, 5, 1, E_levels, L_levels, f_v_levels,
    CPU_capacity, P_idle, P_max, T_total, chain_tasks,
    L_max, E_PU_max, n, m
]

best_solution, best_fitness, chain_activation = genetic_algorithm(
    n=n, m=m, K=K, generations=100, pop_size=200, mutation_rate=0.15, params=params
)




print("\n🔍 遗传算法最佳方案详细分析：")
analyze_solution(best_solution, chain_activation, params)



Generation 1: Best Fitness = -14755.00
Generation 2: Best Fitness = 23445.00
Generation 3: Best Fitness = 37360.00
Generation 4: Best Fitness = 45155.00
Generation 5: Best Fitness = 70405.00
Generation 6: Best Fitness = 96720.00
Generation 7: Best Fitness = 109560.00
Generation 8: Best Fitness = 109560.00
Generation 9: Best Fitness = 109560.00
Generation 10: Best Fitness = 112840.00
Generation 11: Best Fitness = 134305.00
Generation 12: Best Fitness = 134305.00
Generation 13: Best Fitness = 134305.00
Generation 14: Best Fitness = 143955.00
Generation 15: Best Fitness = 156750.00
Generation 16: Best Fitness = 156750.00
Generation 17: Best Fitness = 156750.00
Generation 18: Best Fitness = 156750.00
Generation 19: Best Fitness = 156750.00
Generation 20: Best Fitness = 156750.00
Generation 21: Best Fitness = 156750.00
Generation 22: Best Fitness = 156750.00
Generation 23: Best Fitness = 156750.00
Generation 24: Best Fitness = 156750.00
Generation 25: Best Fitness = 156750.00
Generation 26: